#### https://www.w3cschool.cn/python/

In [159]:
from  jieba import lcut as jc

In [160]:
import json
def printJson(ky):
    print(json.dumps(ky,ensure_ascii=False, encoding='UTF-8'))
   

In [161]:
import jieba.analyse as ja
# 这里使用jieba的textrank提取出_topK个关键词及其比重,默认参数的写法如下 

def getKeywords(_txts,_topK=1000):
      
    _res = ja.textrank(_txts, topK=_topK, withWeight=True)
    
    
    print(_res)

    # 生成关键词比重字典
    _keywords = dict()
    for i in _res:
        _k=i[0]
        _keywords[_k] = i[1]

    printJson(_keywords)
    
    return _keywords



In [162]:
import requests

__url = 'http://www.baidu.com/s?wd='  # 搜索请求网址


def page(word):
    r = requests.get(__url + word)
    if r.status_code == 200:  # 请求错误（不是200）处理
        return r.text
    else:
        print(r.status_code)
        return False


In [178]:
import requests
from bs4 import BeautifulSoup

def init(html):
    # 初始化
    #print help(BeautifulSoup)
    soup = BeautifulSoup(html, 'html5lib')
    results = []

    # 获取结果来源
    result_set = soup.find(id='content_left')  # 结果全显示在页面左边
    result_set = result_set.find_all('div', class_='c-container')  # 结果class固定，其余为硬广

    for i in range(len(result_set)):  # 因为要index所以就用range来
        result = result_set[i]  # 其实就是result_div

        abstract = __get_abstract(result)
           
        results.append(abstract)
    
    return results

# 获取abstract
def __get_abstract(result_div):
    if 'result-op' not in result_div['class']:  # 不是软广
        r_from = result_div.find(class_='c-abstract')
        if not r_from:
            return u'-'
        for em in r_from.find_all('em'):  # 移除abstract中的em标签
            em.unwrap()
        #print 0
       # print r_from.get_text()
        return r_from.get_text()
    else:
        return u'-'  # 其实是因为太麻烦


    

In [164]:
def search(keyword):
    _page = page(keyword)
    results = init(_page)
    
    return results


In [166]:
for ans in answers:
    print ans,_txts.count(ans)

涂经纬 0
撒贝娜 0
任雪 0
李思思 0


In [180]:
def guessAnswer(question,answers):
    _res=search(question)
    #print _res
    _strs=''.join(_res)
    _guess={}
    
    for ans in answers:
        _guess[ans]=_strs.count(ans)
        print ans,_strs.count(ans)
    
    return _guess


## 封装成函数

### 1 简单的统计答案

In [169]:
question=u'下列哪个戏种被称为中国的百戏之祖'
answers=[u'黄梅戏',u'越剧',u'昆曲']

In [171]:
question=u'撒贝宁的姐妹是'
answers=[u'涂经纬',u'撒贝娜',u'任雪',u'李思思']


In [181]:
guessAnswer(question,answers)

涂经纬 0
撒贝娜 4
任雪 0
李思思 0


{u'\u4efb\u96ea': 0,
 u'\u6492\u8d1d\u5a1c': 4,
 u'\u674e\u601d\u601d': 0,
 u'\u6d82\u7ecf\u7eac': 0}

### 2 效果更好的方法

In [101]:
question=u'如果我们长期在很高的氧浓度下呼吸，会？'
answers=[u'A 觉得轻飘飘的',u'B 脑中毒，精神混乱，记忆丧失',u'C 感觉很舒服，增强免疫力']

In [103]:
_res=search(question)
_strs=''.join(_res)

_ans_n={}

for index in range(len(answers)):
    seg_list=jc(answers[index],cut_all=True)
    
    #print seg_list
    for s in seg_list:
        _c=_strs.count(s)
        _ans_n[index]=_ans_n.get(index,0)+_c



for index in range(len(answers)):
    print answers[index],_ans_n[index]


    

A 觉得轻飘飘的 1015
B 脑中毒，精神混乱，记忆丧失 4908
C 感觉很舒服，增强免疫力 2956


In [105]:
def guessAnswer2(question,answers):
    _res=search(question)
    _strs=''.join(_res)

    _ans_n={}

    for index in range(len(answers)):
        seg_list=jc(answers[index],cut_all=True)

        #print seg_list
        for s in seg_list:
            _c=_strs.count(s)
            _ans_n[index]=_ans_n.get(index,0)+_c



    for index in range(len(answers)):
        print answers[index],_ans_n[index]


In [106]:
question=u'西游记里《真假美猴王》一回里，最后两只猴子去到如来那里，如来为了辨别真假想了一个办法，他拿出一个桃子和一个榴莲任其选择，结果一个选了桃子，一个选了榴莲，请问谁才是真的孙悟空？为什么？'
answers=[u'A、选桃子的',u'B、选榴莲的']
guessAnswer2(question,answers)

A、选桃子的 1029
B、选榴莲的 1027


In [108]:
question=u'在工业上，通常用电解盐溶液的方法来制取氯气，而最广泛的不免是"氯碱工业"。请问，氯碱工业中，氯指的是氯气，碱指的是什么呢？'
answers=[u'A、熟碱水',u'B、纯碱',u'C、老碱',u'D、烧碱']
guessAnswer2(question,answers)

A、熟碱水 852
B、纯碱 852
C、老碱 884
D、烧碱 856


## V3 文本关联相似度分析

对于答题这样的项目，首先一个问题，然后有三个答案可以选择，能不能通过分别统计问题与三个答案的关联度来选择出正确的答案，由于数据采集是来自百度的，可能会受到部分广告数据的影响，但是在集合相当大的情况下，关联度还是会呈现正相关。

假设题目是： 

*中国历史上著名的科举制度开始于那个朝代？*
- 汉朝
- 唐朝
- 隋朝

我们先用百度分别搜索`汉朝`，`唐朝`，`隋朝`，得到如下数据：

朝代 | 搜索出的数量（来自百度为您找到相关结果约）
---- | ------------------------------------------
汉朝 | 18600000
唐朝 | 27000000
隋朝 | 14600000

然后我们在用`题目` + 答案的方式，搜索示例：

`中国历史上著名的科举制度开始于那个朝代？ 汉朝` 得到三次的搜索结果：

 关键字  | 搜索出的数量（来自百度为您找到相关结果约）
-------- | ------------------------------------------
Q + 汉朝 | 200000
Q + 唐朝 | 274000
Q + 隋朝 | 220000

关联度计算方式：

``` shell
K = count(Q&A) / (count(Q) * count(A))
```

关联度如下：

答案 | 关联度
---- | ------
汉朝 | 0.0273
唐朝 | 0.0258
隋朝 | 0.0383


In [120]:
question=u'中国历史上著名的科举制度开始于那个朝代'
answers=[u'汉朝',u'唐朝',u'隋朝']
guessAnswer2(question,answers)

汉朝 1
唐朝 6
隋朝 7


In [150]:
question=u'氯碱工业中，氯指的是氯气，碱指的是什么呢？'
answers=[u'A、熟碱水',u'B、纯碱',u'C、老碱',u'D、烧碱']

In [156]:
question=u'撒贝宁的姐妹是'
answers=[u'涂经纬',u'撒贝娜',u'任雪',u'李思思']

In [146]:
import requests
from bs4 import BeautifulSoup

def get_count(html):
    # 初始化
    soup = BeautifulSoup(html, 'html5lib')
    nums =__get_nums(soup)      
          
    return nums

# 获取搜索结果的数量
def __get_nums(result_div):
    _r=result_div.find(class_='nums').get_text()
    return _r
    


In [147]:
def search_countBy(keyword):
    _page = page(keyword)
    results = get_count(_page)
    
    return results


In [123]:
_p=search_countBy(question)

In [126]:
##从字符串提取数字

import re
 
string="A1.45，b5，6.45，8.82"

print re.findall(r"\d+\.?\d*",string)

print ''.join(re.findall(r"\d+\.?\d*",_p))



['1.45', '5', '6.45', '8.82']
3930000


In [148]:
import re
def getNumFromStr(_str):
    _numstr=''.join(re.findall(r"\d+\.?\d*",_str))
    _numstr=_numstr.encode("utf-8")
    _num=float(_numstr)
    return _num

In [157]:
As={}

QAs={}

Q=getNumFromStr(search_countBy(question))

for index in range(len(answers)):
    _strs=question+answers[index]
    _p=search_countBy(_strs)
    _n=getNumFromStr(_p)
    QAs[index]=_n

for index in range(len(answers)):
    _strs=answers[index]
    _p=search_countBy(_strs)
    _n=getNumFromStr(_p)
    As[index]=_n
    
print As,QAs

for index in range(len(QAs)):
    _K=QAs[index]/(Q*As[index])
    print _K

{0: 292000.0, 1: 33700.0, 2: 1740000.0, 3: 5400000.0} {0: 5800.0, 1: 19100.0, 2: 3.0, 3: 22200.0}
2.18274875809e-08
6.22819317181e-07
1.89465706707e-12
4.5177045177e-09


In [158]:
#float(QAs[0])/float(Q*As[0])
#int()
import math

print math.log( 2.18274875809e-08 )
print math.log( 6.22819317181e-07 )
print math.log( 1.89465706707e-12 )
print math.log( 4.5177045177e-09 )

-17.6400957632
-14.2890093808
-26.991983261
-19.2152618222


## END ------------------------

In [13]:
print type(res)
_txts=''.join(res)
print type(_txts)

<type 'list'>
<type 'unicode'>


In [38]:
str2=u'越剧黄梅戏昆曲黄梅戏'
print str2.count(u'黄梅戏')

2


In [35]:
# 统计词频
str1 = u"中文 啊 是 中文 把 哈哈 真的是中文 中文"
print "要统计的句子:"
print str1
print '-----------------------------------'

# 按照空格把句子切分成List
wordList1 = str1.split(None)
print wordList1
# strip any punctuation marks and build modified word list
# start with an empty list
wordList2 = []
for word1 in wordList1:
  # 每个词的最后一个字符
  lastchar = word1[-1:]
    
  print lastchar
        
  # 去除标点符号
  if lastchar in [",", ".", "!", "?", ";"]:
    word2 = word1.rstrip(lastchar)
  else:
    word2 = word1
    
  # 处理后的List2
  wordList2.append(word2.lower())
    
print "处理后的句子:"
print wordList2
print '-----------------------------------'


# 创建一个统计词频的字典
freqD2 = {}

for word in wordList2:
  freqD2[word] = freqD2.get(word, 0) + 1

# 获取关键词，并且排序
keyList = freqD2.keys()
keyList.sort()
print "词频统计结果如下:"
for key in keyList:
 print "%-10s %d" % (key, freqD2[key])

要统计的句子:
中文 啊 是 中文 把 哈哈 真的是中文 中文
-----------------------------------
[u'\u4e2d\u6587', u'\u554a', u'\u662f', u'\u4e2d\u6587', u'\u628a', u'\u54c8\u54c8', u'\u771f\u7684\u662f\u4e2d\u6587', u'\u4e2d\u6587']
文
啊
是
文
把
哈
文
文
处理后的句子:
[u'\u4e2d\u6587', u'\u554a', u'\u662f', u'\u4e2d\u6587', u'\u628a', u'\u54c8\u54c8', u'\u771f\u7684\u662f\u4e2d\u6587', u'\u4e2d\u6587']
-----------------------------------
词频统计结果如下:
中文         3
哈哈         1
啊          1
把          1
是          1
真的是中文      1


In [34]:
p={ 8:2,
    0:1,
    1:2
}

print p.get(0,2)
print p.get(3,9)

print p.keys()
k=p.keys()
print k
k.sort()
print k

1
9
[8, 0, 1]
[8, 0, 1]
[0, 1, 8]


In [14]:
#print printJson(res)
#_txts=''.join(res)
#_kys=getKeywords(_txts,100)

[(u'\u4e2d\u56fd', 1.0), (u'\u767e\u620f', 0.8827857889238552), (u'\u79f0\u4e3a', 0.8291497982774364), (u'\u821e\u8e48', 0.5676589617532118), (u'\u5267\u79cd', 0.47083398897479467), (u'\u6606\u66f2', 0.45634161803502565), (u'\u620f\u66f2', 0.4512751943350847), (u'\u4e3a\u4e3b', 0.3312752803425379), (u'\u620f\u5267', 0.3223009194137544), (u'\u6606\u5c71', 0.30258538956543507), (u'\u8868\u6f14', 0.29873761641016217), (u'\u6587\u5b66\u53f2', 0.28407592872864273), (u'\u827a\u672f', 0.2821143319951763), (u'\u4f53\u7cfb', 0.28073152466487644), (u'\u4eac\u5267', 0.26842595084631726), (u'\u7269\u8d28', 0.26838516933595064), (u'\u6587\u5316\u9057\u4ea7', 0.26747106338531107), (u'\u620f\u79cd', 0.24659667542422972), (u'\u5370\u5ea6', 0.2353710253501502), (u'\u6709\u5173', 0.2263407259956725), (u'\u7ea2\u5305', 0.2263407259956725), (u'\u6d3b\u5316\u77f3', 0.2262766351714662), (u'\u6c5f\u5357', 0.2262766351714662), (u'\u6587\u5316', 0.22529885543214437), (u'\u73b0\u91d1', 0.22529885543214437), (u'

{"达到": 0.17376932428683314, "国花": 0.08343401025689762, "活化石": 0.2262766351714662, "境界": 0.13884628567078108, "戏剧": 0.3223009194137544, "希腊": 0.13291416212240287, "梦幻": 0.16894305159176146, "世界": 0.21090329790701107, "舞蹈": 0.5676589617532118, "揉合": 0.17693055738839736, "现金": 0.22529885543214437, "时间": 0.12322539095028812, "评价": 0.06886822898833475, "为主": 0.3312752803425379, "曲种": 0.10470799664146348, "题材": 0.1256207584475135, "代表作": 0.18292574119316027, "戏曲": 0.4512751943350847, "武术": 0.1207439653833131, "发源": 0.10021185445352968, "戏种": 0.24659667542422972, "唱念": 0.14009210649926881, "置景": 0.1908350369932786, "文学史": 0.28407592872864273, "古希腊": 0.14507024910189606, "有关": 0.2263407259956725, "占有": 0.2127115905892728, "川剧": 0.08835366087651475, "表演": 0.29873761641016217, "纷呈": 0.22130121893313034, "活跃": 0.10062455541269617, "江南": 0.2262766351714662, "飘逸": 0.19574806348049656, "极具": 0.22464346422434706, "中国": 1.0, "文化遗产": 0.26747106338531107, "剧种": 0.47083398897479467, "地位": 0.1230544204084

In [66]:
count = 0

strs=[]

print res

for r in res:
    
    #print('{0} {1} {2} {3} {4}'.format(result.index, result.title, result.abstract, result.show_url, result.url))  # 此处应有格式化输出
    
    a=(r.abstract)
    #strs.append(a)
   
    #a=a.encode("utf-8")
    print type(a)
    print 
    if isinstance(a,str):
        print 'efe'
       # a=a.encode('utf-8')
        
        strs.append(a)
    else:
        
        strs.append(a)
    
  
       # print('{0}'.format(a))   
    #print('{0}'.format(r.abstract))  # 此处应有格式化输出


print type(strs)

sss=str(strs)

#print sss

getKeywords(sss)
   
     

[<__main__.Result object at 0x7f2dcb7609d0>, <__main__.Result object at 0x7f2dcb7e12d0>, <__main__.Result object at 0x7f2dcb7c7e90>, <__main__.Result object at 0x7f2dcb7e1f50>, <__main__.Result object at 0x7f2dcb7f6310>, <__main__.Result object at 0x7f2dcb7f6850>, <__main__.Result object at 0x7f2dcb7f6c50>, <__main__.Result object at 0x7f2dcb7f6fd0>, <__main__.Result object at 0x7f2dcb78f590>, <__main__.Result object at 0x7f2dcb78f3d0>]
<type 'str'>

efe
<type 'unicode'>

<type 'unicode'>

<type 'unicode'>

<type 'unicode'>

<type 'unicode'>

<type 'unicode'>

<type 'unicode'>

<type 'unicode'>

<type 'str'>

efe
<type 'list'>
[]
{}


{}

In [42]:
print('{0}'.format('hello','world'))

In [43]:
a = '\u64cd\u4f5c\u7cfb\u7edf'

print a.decode('unicode-escape').encode('utf-8')

print type(a.decode('unicode-escape').encode('utf-8'))